In [176]:
import pyvisa
import time
import matplotlib.pyplot as plt 
import numpy as np
import csv 
import os
from pyvisa.constants import ControlFlow, Parity, StopBits
import pandas as pd

In [180]:
gpib_address = 7 #for thermo
rm = pyvisa.ResourceManager()
nvm = rm.open_resource(f"GPIB::{gpib_address}", )
print(rm.list_opened_resources())

[<'GPIBInstrument'('GPIB0::7::INSTR')>, <'GPIBInstrument'('GPIB0::7::INSTR')>, <'GPIBInstrument'('GPIB0::7::INSTR')>, <'GPIBInstrument'('GPIB0::7::INSTR')>]


In [178]:
nvm.close()
print(rm.list_opened_resources())

[<'GPIBInstrument'('GPIB0::7::INSTR')>, <'GPIBInstrument'('GPIB0::7::INSTR')>, <'GPIBInstrument'('GPIB0::7::INSTR')>]


In [98]:
while True:
    nvm.write(":TRAC:FEED SENS")
    nvm.write(":INIT")
    data = nvm.write(":READ?")
    
#data2= nvm.query("SENS:CHAN 2")

KeyboardInterrupt: 

In [111]:
nvm.write("*RST")
nvm.write(":ROUT:SCAN (@1,2)")
nvm.write(":TRAC:FEED SENS")
nvm.write(":INIT")
time.sleep(2)
data = nvm.query(":TRAC:DATA?")

In [112]:
data

'+2.09872780E-05,+2.09898859E-05,+2.10010624E-05,+2.10013107E-05,+2.10194415E-05,+2.10214229E-05,+2.10328478E-05,+2.10429066E-05,+2.10347105E-05,+2.10348347E-05,+2.10477498E-05,+2.10492400E-05,+2.10502334E-05,+2.10355798E-05,+2.10604165E-05,+2.10652596E-05,+2.10740686E-05,+2.10616502E-05,+2.10764280E-05,+2.10744411E-05\n'

In [120]:
nvm.write(":SENS:VOLT:DC:RANG AUTO ON")


28

In [171]:
#nvm.write("*RST")
nvm.write(":INIT:CONT OFF")

ch2 = []
ch1 = []
i=0
while i<10:
    nvm.write(":SENS:CHAN 2")
    time.sleep(0.1)
    ch2_reading = float(nvm.query(":READ?"))
    ch2.append(ch2_reading)
    
    nvm.write(":SENS:CHAN 1")
    time.sleep(0.1)
    ch1_reading = float(nvm.query(":READ?"))
    ch1.append(ch1_reading)
    i+=1

print(ch1, ch2)





[-1.97330159e-05, -1.9729042e-05, -1.97417087e-05, -1.97468003e-05, -1.97849228e-05, -1.98052889e-05, -1.98340996e-05, -1.98481323e-05, -1.98858842e-05, -1.98778155e-05] [-0.0185107393, -0.0185107063, -0.0185107543, -0.0185106492, -0.0185106949, -0.0185107018, -0.0185107073, -0.0185106605, -0.0185106696, -0.0185106067]


In [183]:
nvm.write(":INIT:CONT OFF")

ch2 = []
ch1 = []
i=0
while i<3:
    nvm.write(":INIT:CONT OFF")
    nvm.write(":SENS:CHAN 2")
    time.sleep(0.1)
    ch2_reading = float(nvm.query(":READ?"))
    ch2.append(ch2_reading)

    nvm.write(":INIT:CONT ON")
    nvm.write(":SENS:CHAN 1")
    nvm.write("STAT:PRES;*CLS")
    nvm.write("STAT:MEAS:ENAB 512")
    nvm.write("*SRE 1")
    nvm.write("TRIG:COUN 10")
    nvm.write(":TRAC:POIN 10")
    nvm.write(":TRAC:FEED SENS1")
    nvm.write(":TRAC:FEED:CONT NEXT")
    #nvm.write(":INIT")
    time.sleep(1)

    raw = nvm.query("TRAC:DATA?")
    ch1.append(np.mean(raw))

    i+=1

print(ch2, raw, ch1)

        

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U160'), dtype('<U160')) -> None

In [153]:
nvm.write("*RST")
nvm.write(":INIT:CONT OFF")         # Manual mode
nvm.write(":TRAC:CLE")              # Clear trace buffer
nvm.write(":TRAC:POIN 10")          # Allocate buffer for 10 points
nvm.write(":TRAC:FEED SENS")        # Feed raw readings
nvm.write(":TRAC:FEED:CONT NEXT")   # Feed only on next INIT
nvm.write(":TRIG:COUN 10")          # Take 10 readings
nvm.write(":SENS:CHAN 1")           # Select channel 1 (or 2)
nvm.write(":INIT")                  # Start acquisition
nvm.query("*OPC?")                  # Wait until done

# Retrieve data from buffer
time.sleep(5)
raw = nvm.query(":TRAC:DATA?")
readings = np.array([float(x) for x in raw.strip().split(',')])
average = np.mean(readings)

print("Readings:", readings)
print("Average:", average)

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [146]:
ch2, ch1

([0.0185850021, 0.0185850654, 0.0185850203],
 [2.1079629e-05, 2.10840952e-05, 2.10279647e-05])

In [139]:
nvm.write("*RST")
nvm.write(':SENS:VOLT:DC:DFIL:COUN 10')

28

In [119]:
nvm.write("*RST")
nvm.write("FORM:DATA ASC")
nvm.write("CONF:VOLT") #set voltage mode
nvm.write(":TRAC:CLE") #clear buffer
nvm.write(":TRAC:POIN 10") #make 10 readings to get a 1Hz measurement
nvm.write(":TRAC:FEED SENS") #store raw readings
nvm.write(":INIT:CONT ON") #cont operation on
nvm.write(":TRAC:FEED:CONT NEXT")
nvm.write(":CALC2:STAT ON") #so I can use average
nvm.write("INIT") #start measurement
nvm.query("*OPC?") #wait until done
raw = nvm.query(":TRAC DATA")
values = nvm.query(":CALC2:DATA?")


VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [10]:
raw


'9.910000E+37\n'

In [8]:
nvm.query("*IDN")

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [9]:
nvm.close()

In [17]:
nvm.write("*RST")
nvm.write(":TRAC:CLE") #clear buffer
nvm.write("SAMPLE:COUN 10")
nvm.write("FORM:DATA ASC")
data =nvm.write("READ?")






In [33]:
nvm.write("*RST")
nvm.write("STAT:PRES;*CLS")
nvm.write("STAT:MEAS:ENAB 512")
nvm.write("*SRE 1")

nvm.write("TRIG:COUN 10")

nvm.write(":TRAC:POIN 10")
nvm.write(":TRAC:FEED SENS1") 
nvm.write(":TRAC:FEED:CONT NEXT")

nvm.write(":INIT")

nvm.query("*OPC?") 

status = nvm.query("STAT:MEAS?")
nvm.write("FORM:ELEM READ, UNIT")
readings = nvm.query("TRAC:DATA?")



VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [165]:
nvm.write("*RST")
nvm.write("STAT:PRES;*CLS")
nvm.write("STAT:MEAS:ENAB 512")
nvm.write("*SRE 1")

nvm.write("TRIG:COUN 10")

nvm.write(":TRAC:POIN 10")
nvm.write(":TRAC:FEED SENS1") 
nvm.write(":TRAC:FEED:CONT NEXT")

nvm.write(":INIT")

time.sleep(5)

raw = nvm.query("TRAC:DATA?")
print(raw)

InvalidSession: Invalid session handle. The resource might be closed.

In [79]:
nvm.write("*RST")
nvm.write("STAT:PRES;*CLS")
nvm.write("STAT:MEAS:ENAB 512")
nvm.write("*SRE 1")

#for both channels
nvm.write("TRIG:COUN 20")


nvm.write(":TRAC:POIN 20")
nvm.write(":TRAC:FEED SENS1") 
nvm.write(":TRAC:FEED:CONT NEXT")

nvm.write("ROUT:SCAN:INT 2")

nvm.write(":INIT")

time.sleep(10)

raw = nvm.query("TRAC:DATA?")
print(raw)

-4.72888468E-02,-6.83147986E-02,-8.95336806E-02,-1.10450136E-01,+7.06938431E-03,-3.49501310E-02,-5.57495210E-02,-7.65735369E-02,-9.76226939E-02,-1.18204664E-01,+3.51120894E-03,-3.85181863E-02,-5.91756670E-02,-7.94383836E-02,-1.00027958E-01,-1.37526396E-05,-1.37526396E-05,-1.37526396E-05,-1.37526396E-05,-1.37526396E-05



In [62]:
time.sleep(1)

In [74]:
raw = nvm.query("TRAC:DATA?")

In [75]:
raw

'-4.30223562E-02,-6.55555278E-02,-8.81646557E-02,-1.10729783E-01,+1.12813345E-02,-3.37526022E-02,-5.61145860E-02,-7.84046183E-02,-1.00616757E-01,+1.14036167E-02\n'

In [53]:
raw

'+4.30390515E-05,+4.30601629E-05,+4.30633917E-05,+4.30702219E-05,+4.30647578E-05,+4.30758013E-05,+4.30726967E-05,+4.30802720E-05,+4.30802720E-05,+4.30802720E-05\n'

In [32]:
nvm.write("*RST")
nvm.write(":INIT:CONT OFF")
nvm.write(":TRAC:CLE") #clear buffer
nvm.write(":TRAC:POIN 10") #make 10 readings to get a 1Hz measurement
nvm.write(":TRAC:FEED SENS") #store raw readings
nvm.write(":TRAC:FEED:CONT NEXT")
nvm.write(":INIT")
nvm.write("*WAI") 
data =nvm.query(":TRAC DATA?")

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [26]:
data

13

In [ ]:
nvm.write("FORM:DATA ASC")
nvm.write("CONF:VOLT") #set voltage mode
nvm.write(":TRAC:CLE") #clear buffer
nvm.write(":TRAC:POIN 10") #make 10 readings to get a 1Hz measurement
nvm.write(":TRAC:FEED SENS") #store raw readings
nvm.write(":INIT:CONT ON") #cont operation on
nvm.write(":TRAC:FEED:CONT NEXT")
nvm.write(":CALC2:STAT ON") #so I can use average
nvm.write("INIT") #start measurement
nvm.query("*OPC?") #wait until done
raw = nvm.query(":TRAC DATA?")
values = nvm.query(":CALC2:DATA?")

In [31]:
nvm.write("*RST")                            # Reset the instrument
nvm.write(":TRAC:CLE")                       # Clear the trace buffer
nvm.write(":TRAC:POIN 10")                   # Set number of points
nvm.write(":TRAC:FEED SENS")                 # Feed raw sensor readings to buffer
nvm.write(":TRAC:FEED:CONT NEXT")            # Continue feeding buffer on next trigger
nvm.write(":TRIG:COUN 10")                   # Set number of trigger counts
nvm.write(":INIT")                           # Start measurement
nvm.query("*OPC?")                           # Wait until operation is complete

data = nvm.query(":TRAC:DATA?")              # Now data should be ready
print("Raw data:", data)

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.